In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedShuffleSplit

pd.options.mode.chained_assignment = None

In [ ]:
df_base = pd.read_csv('../data/waybill_base.csv')
display(df_base.head())

In [ ]:
df_GDP = pd.read_csv('../data/GDP-all.csv')
display(df_GDP.head())

In [ ]:
df_people = pd.read_csv('../data/people-all.csv')
display(df_people.head())

In [ ]:
df_city_ratio = pd.read_excel('../data/socio_economic_data.xlsx')
display(df_city_ratio.head())

In [ ]:
'''
Add two new columns after df_base: GDP and people
'''
import json

with open('../data/city_name_list.json', 'r') as f:
    city_name_list = json.load(f)
df_base['GDP_2022'] = 0
df_base['people_2022'] = 0
df_base['city_ratio'] = 0
year_list = [2022,2021,2020]

for i in range(len(df_base)):
    city_name = df_base.loc[i,'city']
    city_serach_name = city_name
    for temp_list in city_name_list:
        if city_name in temp_list:
            repeat_list = temp_list
            for city_temp in repeat_list:
                if city_temp in df_GDP['rev_addr_city_id'].values:
                    city_serach_name = city_temp
                    # print(city_serach_name)
    flag = False
    for year in year_list:
        if len(df_GDP[(df_GDP['rev_addr_city_id']==city_serach_name) & (df_GDP['year']==year)]) == 1:
            index = df_GDP[(df_GDP['rev_addr_city_id']==city_serach_name) & (df_GDP['year']==year)].index[0]
            GDP = df_GDP.at[index,'gdp']
            df_base.loc[i,'GDP_2022'] = GDP
            flag = True
            break
    if not flag:
        print(f'error GDP:{city_name}')

for i in range(len(df_base)):
    city_name = df_base.loc[i,'city']
    city_serach_name = city_name
    for temp_list in city_name_list:
        if city_name in temp_list:
            repeat_list = temp_list
            for city_temp in repeat_list:
                if city_temp in df_people['rev_addr_city_id'].values:
                    city_serach_name = city_temp
                    # print(city_serach_name)
    flag = False
    for year in year_list:
        if len(df_people[(df_people['rev_addr_city_id']==city_serach_name) & (df_people['year']==year)]) == 1:
            index = df_people[(df_people['rev_addr_city_id']==city_serach_name) & (df_people['year']==year)].index[0]
            people = df_people.at[index,'population']
            df_base.loc[i,'people_2022'] = people
            flag = True
            break
    if not flag:
        print(f'error people:{city_name}')

df_city_ratio = pd.read_excel('../data/socio_economic_data.xlsx')
for i in range(len(df_base)):
    city_name = df_base.loc[i,'city']
    city_serach_name = city_name
    for temp_list in city_name_list:
        if city_name in temp_list:
            repeat_list = temp_list
            for city_temp in repeat_list:
                if city_temp in df_city_ratio['city'].values:
                    city_serach_name = city_temp
                    # print(city_serach_name)
    flag = False
    for year in [2021,2020]:
        if len(df_city_ratio[(df_city_ratio['city']==city_serach_name) & (df_city_ratio['year']==year)]) == 1:
            index = df_city_ratio[(df_city_ratio['city']==city_serach_name) & (df_city_ratio['year']==year)].index[0]
            city_ratio = df_city_ratio.at[index,'urbanization(%)']
            df_base.loc[i,'city_ratio'] = city_ratio
            flag = True
            break
    if not flag:
        print(f'error city_ratio:{city_name}')

display(df_base.head())
df_base.to_csv('../data/waybill_base_20240829.csv')

In [ ]:
df = pd.read_csv('../data/Ce_filter_20240823.csv')
df['sum_Ce_2023_01'],df['sum_Ce_2023_07'],df['sum_Ce_2024_01'] = 0,0,0
company_list = ['JD','SF','YT','YD','ZT']
month_list = ['2023_01','2023_07','2024_01']

columns_list = [] # Stores columns that need to be guaranteed to be non-0
for month in month_list:
    for company in company_list:
        columns_list.append(f'sum_Ce_{company}_{month}')
        df[f'sum_Ce_{month}'] += df[f'sum_Ce_{company}_{month}']
    df[f'sum_Ce_{month}'] *= 30 # Multiply by the number of days in a month
    df[f'sum_Ce_{month}'] /= 1000 # multiply t

df['has_zero'] = df[columns_list].eq(0).any(axis=1) 
for month in month_list:
    df.loc[df['has_zero'], 'sum_Ce_2023_01'] = 0
    df.loc[df['has_zero'], 'sum_Ce_2023_07'] = 0
    df.loc[df['has_zero'], 'sum_Ce_2024_01'] = 0

print(len(df))
df = df[df['sum_Ce_2024_01']!= 0]
print(len(df))


In [ ]:
df_merge = pd.merge(df_base,df[['city','sum_Ce_2024_01']],on='city',how='inner')
print(len(df_merge))
display(df_merge.head())

In [ ]:
df = pd.read_csv('../data/distance_filter_20240822.csv')
df['sum_distance_2023_01'],df['sum_distance_2023_07'],df['sum_distance_2024_01'] = 0,0,0
company_list = ['JD','SF','YT','YD','ZT']
month_list = ['2023_01','2023_07','2024_01']

columns_list = [] 
for month in month_list:
    for company in company_list:
        columns_list.append(f'sum_distance_{company}_{month}')
        df[f'sum_distance_{month}'] += df[f'sum_distance_{company}_{month}']
    df[f'sum_distance_{month}'] *= 30 

df['has_zero'] = df[columns_list].eq(0).any(axis=1) 
for month in month_list:
    df.loc[df['has_zero'], 'sum_distance_2023_01'] = 0
    df.loc[df['has_zero'], 'sum_distance_2023_07'] = 0
    df.loc[df['has_zero'], 'sum_distance_2024_01'] = 0

print(len(df))
df = df[df['sum_distance_2024_01']!= 0]
print(len(df))
df_merge = pd.merge(df_merge,df[['city','sum_distance_2024_01']],on='city',how='inner')
display(df_merge.head())

In [ ]:
def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response,
                                       ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop(0)
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model

In [ ]:
from statsmodels.iolib.summary2 import summary_col

# LInear Regression
df_merge = df_merge.dropna(subset=['city_level_num','GDP_2022','people_2022','sum_distance_2024_01','city_ratio','elevator_ratio'], how='any')
print(len(df_merge))
df_merge = df_merge[['city_ratio','city_level_num','GDP_2022','people_2022','elevator_ratio','sum_distance_2024_01','sum_Ce_2024_01']]
response = 'sum_Ce_2024_01'
model = forward_selection(df_merge, response)
print(model.summary())

rounded_pvalues = model.pvalues.round(16)

print("P-values:", rounded_pvalues)

In [ ]:
from statsmodels.iolib.summary2 import summary_col

# OLS Regression
df_merge = df_merge.dropna(subset=['city_level_num','GDP_2022','people_2022','sum_distance_2024_01','city_ratio','elevator_ratio'], how='any')
print(len(df_merge))
df_merge['average_people_GDP_2022'] = df_merge['GDP_2022']/df_merge['people_2022']
X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
Y = df_merge['sum_Ce_2024_01']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

rounded_pvalues = results.pvalues.round(100)

print("P-values:", rounded_pvalues)

In [ ]:
# RLM
df_merge = df_merge.dropna(subset=['city_level_num','GDP_2022','people_2022','sum_distance_2024_01','city_ratio','elevator_ratio'], how='any')
print(len(df_merge))
df_merge['average_people_GDP_2022'] = df_merge['GDP_2022']/df_merge['people_2022']
X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
Y = df_merge['sum_Ce_2024_01']
X = sm.add_constant(X)
model = sm.RLM(Y,X)
results = model.fit()
print(results.summary())

rounded_pvalues = results.pvalues.round(100)

print("P-values:", rounded_pvalues)

In [ ]:
# GLS
df_merge = df_merge.dropna(subset=['city_level_num','GDP_2022','people_2022','sum_distance_2024_01','city_ratio','elevator_ratio'], how='any')
print(len(df_merge))
df_merge['average_people_GDP_2022'] = df_merge['GDP_2022']/df_merge['people_2022']
X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
Y = df_merge['sum_Ce_2024_01']
X = sm.add_constant(X)
model = sm.GLS(Y,X)
results = model.fit()
print(results.summary())

rounded_pvalues = results.pvalues.round(100)

print("P-values:", rounded_pvalues)

In [ ]:
from sklearn.linear_model import Lasso
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import train_test_split

df_merge = df_merge.dropna(subset=['city_level_num','GDP_2022','people_2022','sum_distance_2024_01','city_ratio','elevator_ratio'], how='any')
print(len(df_merge))
df_merge['average_people_GDP_2022'] = df_merge['GDP_2022']/df_merge['people_2022']
X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
Y = df_merge['sum_Ce_2024_01']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Lasso model
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)
lasso_coefficients = lasso_model.coef_

print("Lasso Coefficients:")
nonzero_features = X.columns[lasso_coefficients!= 0]
X_ols = sm.add_constant(X_train[nonzero_features])
ols_model = OLS(y_train, X_ols).fit()

print("\nOLS Regression Results for Non-Zero Features:")
print(ols_model.summary())

rounded_pvalues = ols_model.pvalues.round(100)

print("P-values:", rounded_pvalues)

In [ ]:
# xgboost
max_r2 = 0
X = df_merge[['city_level_num','GDP_2022','people_2022','average_people_GDP_2022','sum_distance_2024_01','city_ratio']]
y = df_merge['sum_Ce_2024_01']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)
for colsample_bytree in [0.1, 0.2, 0.5, 0.7, 0.9]:
    for learning_rate in [0.001, 0.005, 0.01, 0.05, 0.1]:
        for max_depth in [3, 5, 7, 9]:
            for alpha in [0.1, 0.3, 0.5, 0.7, 1]:
                for n_estimators in [1, 3, 5, 8, 10]:
                    model = xgb.XGBRegressor(objective='reg:squarederror',colsample_bytree=colsample_bytree,learning_rate=learning_rate,\
                                             max_depth=max_depth,alpha=alpha,n_estimators=n_estimators)
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    r2 = r2_score(y_test, y_pred)
                    if r2 > max_r2:
                        max_r2 = r2
print(max_r2)

In [ ]:
# xgboost
X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
y = df_merge['sum_Ce_2024_01']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)
model = xgb.XGBRegressor()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_val_score(model, X, Y, cv=kf, scoring='r2')
for result in cv_results:
    print(result)
print(f'Cross-validated R^2: {cv_results.mean()}') 

In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df_merge[['sum_distance_2024_01','elevator_ratio','people_2022']]
y = df_merge['sum_Ce_2024_01']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
model = RandomForestRegressor(n_estimators=20, max_depth=None, random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_val_score(model, X, Y, cv=kf, scoring='r2')
for result in cv_results:
    print(result)
print(f'Cross-validated R^2: {cv_results.mean()}') 